# CS246 - Colab 7
## Decision Trees in Spark

In [1]:
from IPython.display import Image
print("Colab 7 Mascot")
Image(url='https://media2.giphy.com/media/kudIERso2pFiE/giphy.gif?cid=ecf05e47bipq7szjsc8vmss2uxmt1zha9gksgdrn1z8mmadj&ep=v1_gifs_search&rid=giphy.gif&ct=g',width=200)

Colab 7 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=8508b8484bbe55bdae79d2be7a246a0788e34dd0e019e42d476ce648938d1e35
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1aJrdYMVmmnUKYhLTlXtyB0FQ9gYJqCrs'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-train.txt')

id='1yLwxRaJIyrC03yxqbTKpedMmHEF86AAq'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-test.txt')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [6]:
# create the session
conf = SparkConf() \
    .set("spark.ui.port", "4050") \
    .set("spark.driver.cores", "4") \
    .set("spark.driver.memory", "16g") \
# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

![MNIST](https://upload.wikimedia.org/wikipedia/commons/thumb/2/27/MnistExamples.png/220px-MnistExamples.png)

In this Colab, we will be using the famous [MNIST database](https://en.wikipedia.org/wiki/MNIST_database), a large collection of handwritten digits that is widely used for training and testing in the field of machine learning.

For your convenience, the dataset has already been converted to the popular LibSVM format, where each digit is represented as a sparse vector of grayscale pixel values.

In [7]:
training = spark.read.format("libsvm").load("mnist-digits-train.txt")
test = spark.read.format("libsvm").load("mnist-digits-test.txt")

# Cache data for multiple uses
training.cache()
test.cache()

DataFrame[label: double, features: vector]

In [8]:
training.show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
training.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [10]:
test.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



### Your task

First of all, find out how many instances(samples) we have in our training / test split.

In [12]:
print("Number of samples in training: ", training.count())
print("Number of samples in test: ", test.count())

Number of samples in training:  60000
Number of samples in test:  10000


Now train a Decision Tree on the training dataset using Spark MLlib.

You can refer to the Python example on this documentation page: [https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier](https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier)

(Note : You **don't** need to and should **NOT** use Indexer as shown by the example, so that your answers can match our solution answers for Gradescope problems.)

Use **seed of 7** (colab 7) for training purposes throughout the colab.

In [23]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", seed=7)
model = dt.fit(training)

With the Decision Tree you just induced on the training data, predict the labels of the test set.
Print the predictions for the first 10 digits, and compare them with the labels.

In [24]:
predictions = model.transform(test)
predictions.select("prediction", "label", "features").show(10)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       7.0|  7.0|(778,[202,203,204...|
|       6.0|  2.0|(778,[94,95,96,97...|
|       1.0|  1.0|(778,[128,129,130...|
|       0.0|  0.0|(778,[124,125,126...|
|       9.0|  4.0|(778,[150,151,159...|
|       1.0|  1.0|(778,[156,157,158...|
|       5.0|  4.0|(778,[149,150,151...|
|       9.0|  9.0|(778,[179,180,181...|
|       6.0|  5.0|(778,[129,130,131...|
|       9.0|  9.0|(778,[209,210,211...|
+----------+-----+--------------------+
only showing top 10 rows



The small sample above looks good, but not great!

Let's dig deeper. Compute the accuracy of our model, using the ```MulticlassClassificationEvaluator``` from MLlib.

In [25]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % accuracy)

Test Accuracy = 0.678 


Find out the max depth of the trained Decision Tree, and its total number of nodes.

In [26]:
print("Max depth: ", dt.getMaxDepth())
print("Number of nodes: ", model.numNodes)

Max depth:  5
Number of nodes:  61


It appears that the default settings of the Decision Tree implemented in MLlib did not allow us to train a very powerful model!

Before starting to train a Decision Tree, you can tune the max depth it can reach using the [setMaxDepth()](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.DecisionTreeClassifier.html#pyspark.ml.classification.DecisionTreeClassifier.setMaxDepth) method. Train 21 different DTs, varying the max depth from 0 to 20, endpoints included (i.e., [0, 20]). For each value of the parameter, print the accuracy achieved on the test set, and the number of nodes contained in the given DT.

For this problem, we say that the decision trees overfit the dataset if the classification accuracy starts to drop.

**IMPORTANT:** this parameter sweep can take 30 minutes or more, depending on how busy is your Colab instance. Notice how the induction time grows super-linearly!

In [27]:
for i in range(21):
  dt.setMaxDepth(i)
  model = dt.fit(training)
  predictions = model.transform(test)
  evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
  accuracy = evaluator.evaluate(predictions)
  print("Max depth: ", i)
  print("Number of nodes: ", model.numNodes)
  print("Accuracy: ", accuracy)

Max depth:  0
Number of nodes:  1
Accuracy:  0.1135
Max depth:  1
Number of nodes:  3
Accuracy:  0.1994
Max depth:  2
Number of nodes:  7
Accuracy:  0.3442
Max depth:  3
Number of nodes:  15
Accuracy:  0.4939
Max depth:  4
Number of nodes:  31
Accuracy:  0.5942
Max depth:  5
Number of nodes:  61
Accuracy:  0.678
Max depth:  6
Number of nodes:  121
Accuracy:  0.7445
Max depth:  7
Number of nodes:  245
Accuracy:  0.7852
Max depth:  8
Number of nodes:  489
Accuracy:  0.8194
Max depth:  9
Number of nodes:  929
Accuracy:  0.8492
Max depth:  10
Number of nodes:  1677
Accuracy:  0.8664
Max depth:  11
Number of nodes:  2683
Accuracy:  0.8721
Max depth:  12
Number of nodes:  3749
Accuracy:  0.8768
Max depth:  13
Number of nodes:  4683
Accuracy:  0.8791
Max depth:  14
Number of nodes:  5513
Accuracy:  0.8804
Max depth:  15
Number of nodes:  6167
Accuracy:  0.8797
Max depth:  16
Number of nodes:  6617
Accuracy:  0.8801
Max depth:  17
Number of nodes:  6879
Accuracy:  0.8791
Max depth:  18
Number 

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!